In [577]:
import pymysql
import pandas as pd
import numpy as np

In [578]:
try:
    conn=pymysql.connect(host='localhost', user='root', passwd='', db='unifin')
    cur=conn.cursor()
    query="select a.id_c,op.date_entered,op2.vigencialinea_c FROM accounts_cstm a,opportunities op,opportunities_cstm op2,accounts_opportunities ao WHERE a.id_c=ao.account_id AND op2.id_c=ao.opportunity_id and op2.id_c=op.id and op.deleted=0;"
    cur.execute(query)
    res = cur.fetchall()
    cur.close()
    conn.close()
    
except pymysql.Error as e:
    msj= ("Error %d: %s" % (e.args[0], e.args[1]))
    print(msj)
else:
    headers=['id','Fecha','Vigencia']
    dataset_dummy={}
    filas=0

    for h in headers:
        dataset_dummy[h]=[]

    if(len(res)>0):
        for r in res:
            filas+=1
            for i in range(len(headers)):
                dataset_dummy[headers[i]].append(r[i])

    print('El numero de filas de este dataset es de:'+str(filas))

    df1=pd.DataFrame(dataset_dummy) 
    print(df1.shape)

    print(df1.dtypes)

El numero de filas de este dataset es de:30064
(30064, 3)
id                  object
Fecha       datetime64[ns]
Vigencia            object
dtype: object


## Se crea columna de frecuencia

In [579]:
df2=df1.groupby('id').count()
df2=df2.drop(['Fecha'],axis=1)
df2= df2.rename(columns={'Vigencia': 'Frecuencia'})
df2.head()

,Frecuencia
id,
0006f043-b037-4d15-a48a-06f0f26a2869,1
000f58b7-a684-44c9-94c9-3437627f90f1,2
001b6d7c-97ad-4ff8-9475-37a8fc1f7582,1
001d6010-634b-484e-b752-6479de28ea47,2
00205105-2810-4e84-b0e6-aad0abd2a1e0,1


## Se añade columna de frecuencia

In [580]:
df3=df1.merge(df2,on=['id'])
df3.head()    

,id,Fecha,Vigencia,Frecuencia
0,e45d4e71-22c6-421d-85ec-77ecaead04e9,2015-07-31,0000-00-00,780
1,e45d4e71-22c6-421d-85ec-77ecaead04e9,2012-02-21,0000-00-00,780
2,e45d4e71-22c6-421d-85ec-77ecaead04e9,2013-06-13,0000-00-00,780
3,e45d4e71-22c6-421d-85ec-77ecaead04e9,2014-11-07,0000-00-00,780
4,e45d4e71-22c6-421d-85ec-77ecaead04e9,2015-08-14,0000-00-00,780


## Se convierte fecha a timestamp

In [555]:
df3['ts'] = df3.Fecha.values.astype(np.int64) // 10 ** 9
print(df3.dtypes)
df3.head()

id                    object
Fecha         datetime64[ns]
Producto              object
Frecuencia             int64
ts                     int64
dtype: object


,id,Fecha,Producto,Frecuencia,ts
0,b9d12fcc-5280-4a87-838f-672d00e70088,1999-05-15,1,1,926726400
1,1c10b761-1dec-4112-980e-8886c8edab63,1999-10-15,1,1,939945600
2,0035ce7c-fd31-4c02-af7a-92d80a6e10e2,2000-01-15,1,1,947894400
3,17fd62b4-983c-495d-9d22-70b914d89774,2000-06-25,1,1,961891200
4,6302a99f-85ab-4954-95cc-b21561ff89f4,2000-08-01,1,1,965088000


## Se filtran ids con frecuencia=2

In [556]:
df4=df3[df3['Frecuencia']==2]
df4.dtypes

id                    object
Fecha         datetime64[ns]
Producto              object
Frecuencia             int64
ts                     int64
dtype: object

## Se crea nuevo df separando fechas 1 y 2

In [557]:
df4=df4.sort_values(by=['id','ts'],ascending=True)
df4= df4.reset_index(drop=True)
print(df4.shape)
df4.head(10)

(6428, 5)


,id,Fecha,Producto,Frecuencia,ts
0,000f58b7-a684-44c9-94c9-3437627f90f1,2015-07-21 00:00:00,1,2,1437436800
1,000f58b7-a684-44c9-94c9-3437627f90f1,2017-08-25 00:11:04,1,2,1503619864
2,001b6d7c-97ad-4ff8-9475-37a8fc1f7582,2017-06-29 15:26:44,1,2,1498750004
3,001b6d7c-97ad-4ff8-9475-37a8fc1f7582,2017-06-29 21:48:18,1,2,1498772898
4,002b7e10-62d1-11e8-b155-00155d967307,2018-05-29 16:59:33,1,2,1527613173
5,002b7e10-62d1-11e8-b155-00155d967307,2018-05-29 16:59:33,1,2,1527613173
6,004926e1-4e90-4409-9aca-e5a47f4eeddf,2013-03-05 12:41:05,3,2,1362487265
7,004926e1-4e90-4409-9aca-e5a47f4eeddf,2016-12-15 23:04:48,3,2,1481843088
8,00713ebb-6e9f-4a0c-be22-a70e891e1e6c,2010-05-06 00:00:00,1,2,1273104000
9,00713ebb-6e9f-4a0c-be22-a70e891e1e6c,2014-10-30 17:02:10,1,2,1414688530


In [558]:
#obj={'id':[],'Fecha1':[],'Fecha2':[],'Producto1':[],'Producto2':[]}
obj={'id':[],'Fecha1':[],'Fecha2':[]}
filas=0
temp='xxx'

for index,row in df4.iterrows():
    if(row['id']==temp):
        obj['Fecha'+str(x)].append(row['ts'])
        #obj['Producto2'].append(row['Producto'])
        x=x+1
        filas=filas+1
        continue
        
    temp=row['id']
    
    obj['id'].append(row['id'])
    #obj['Producto1'].append(row['Producto'])
    obj['Fecha1'].append(row['ts'])
    x=2

print('El numero de filas de este dataset es de:'+str(filas))

df5=pd.DataFrame(obj) 
print(df5.shape)
df5.head(10)

El numero de filas de este dataset es de:3214
(3214, 3)


,id,Fecha1,Fecha2
0,000f58b7-a684-44c9-94c9-3437627f90f1,1437436800,1503619864
1,001b6d7c-97ad-4ff8-9475-37a8fc1f7582,1498750004,1498772898
2,002b7e10-62d1-11e8-b155-00155d967307,1527613173,1527613173
3,004926e1-4e90-4409-9aca-e5a47f4eeddf,1362487265,1481843088
4,00713ebb-6e9f-4a0c-be22-a70e891e1e6c,1273104000,1414688530
5,00ab3c06-fd56-4865-94e5-9bef9bd95ae7,1392810741,1392988592
6,00b8d522-0759-4458-85d7-364c5e8ebac1,1404172800,1446744355
7,00cb5a10-9b38-446d-bb71-a46c00c8845f,1462283775,1464361361
8,00d8e5fa-426a-11e8-89ad-00155d967307,1523988404,1527520129
9,01289ec3-163a-4bab-a30d-4d68b3b95a44,1393804800,1432166400


In [559]:
#df5[df5['Producto1']!=df5['Producto2']]

## Se comienza el modelo

In [560]:
y_header='Fecha2'
drop_col=['id',y_header]
Y=df5[y_header]
X=df5.drop(drop_col,axis=1)
#X.head()

In [561]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2)

In [562]:
from math import sqrt
from sklearn.metrics import mean_squared_error 

## Regresion Lineal

In [563]:
from sklearn.linear_model import LinearRegression

lr=LinearRegression()

lr.fit(X_train,Y_train)

scorel=lr.score(X_test,Y_test)

predictl=lr.predict(X_test)

errorl = sqrt(mean_squared_error(Y_test,predictl))

## Random Forest

%%time

from sklearn.ensemble import RandomForestRegressor

forest=RandomForestRegressor(n_jobs=2, n_estimators=40)

forest.fit(X_train,Y_train)

scoref=forest.score(X_test,Y_test)

predictf=forest.predict(X_test)

errorf = sqrt(mean_squared_error(Y_test,predictf))

## SVM

%%time

from sklearn.svm import SVR
from sklearn.svm import SVC

C=1e3 
svc_rbf = SVR(kernel="rbf", C=C, gamma=0.1) #kernel radial

model= svc_rbf.fit(X_train,Y_train)

scores=model.score(X_test,Y_test)

predicts=model.predict(X_test)

errors = sqrt(mean_squared_error(Y_test,predicts))

## KNN

In [564]:
%%time

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_jobs=2,n_neighbors =20)

knn.fit(X_train, Y_train)

scorek=knn.score(X_test,Y_test)

predictknn=knn.predict(X_test)

errork = sqrt(mean_squared_error(Y_test,predictknn))

#predictknn2=knn.predict_proba(X_test)

CPU times: user 128 ms, sys: 12.2 ms, total: 140 ms
Wall time: 204 ms


## Conclusion

In [565]:
#----------------------------

print('>> LR.... <<')
print('Score',scorel)
print('Error',errorl)
print(list(zip(predictl, Y_test))[:5])

#----------------------------

#print('>> Forest.... <<')
#print('Score',scoref)
#print('Error',errorf)
#print(list(zip(predictf, Y_test))[:5])
#----------------------------

#print('>> SVR.... <<')
#print('Score',scores)
#print('Error',errors)
#print(list(zip(predicts, Y_test))[:5])

#----------------------------

print('>> KNN.... <<')
print('Score',scorek)
print('Error',errork)
print(list(zip(predictknn, Y_test))[:5])
  

>> LR.... <<
Score 0.7408272177277959
Error 41687651.03975593
[(1497253610.1159387, 1489711738), (1491316583.179412, 1481213177), (1505613541.5031962, 1502320366), (1482176805.8308558, 1503936106), (1350350054.8535967, 1294272000)]
>> KNN.... <<
Score 0.7245631729019608
Error 42975776.529958606
[(1503399817.7, 1489711738), (1490075843.8, 1481213177), (1509427972.4, 1502320366), (1487313420.3, 1503936106), (1329806907.5, 1294272000)]


In [566]:
df={}
df=pd.DataFrame(df)

#df['Fecha1']=X_test['Fecha1']
df['Y_test']=Y_test
df['Predict']=predictknn

#df['Fecha1']=pd.to_datetime(df['Fecha1'], unit='s')
df['Y_test']=pd.to_datetime(df['Y_test'], unit='s')
df['Predict']=pd.to_datetime(df['Predict'], unit='s')
df.head(10)

,Y_test,Predict
1011,2017-03-17 00:48:58,2017-08-22 11:03:37.700000048
447,2016-12-08 16:06:17,2017-03-21 05:57:23.799999952
908,2017-08-09 23:12:46,2017-10-31 05:32:52.400000095
809,2017-08-28 16:01:46,2017-02-17 06:37:00.299999952
1897,2011-01-06 00:00:00,2012-02-21 06:48:27.500000000
2734,2013-04-18 09:15:18,2014-09-04 12:51:10.250000000
3032,2015-02-05 19:30:58,2012-11-02 00:13:19.299999952
401,2018-03-13 00:02:35,2016-09-04 11:06:17.650000095
1743,2011-03-18 00:00:00,2012-08-16 10:00:42.099999905
1590,2015-08-20 17:55:32,2015-06-01 12:57:50.150000095


## Guardar Modelo

In [254]:
import pickle

filename = 'agenda3_knn_1var.sav'
pickle.dump(lr, open(filename, 'wb'))

## Cargar Modelo

In [567]:
import pickle

filename = 'agenda1_knn_1var.sav'
#filename = 'agenda2_lr_1var.sav'

model = pickle.load(open(filename, 'rb'))

score=model.score(X_test,Y_test)

predict=model.predict(X_test)

error = sqrt(mean_squared_error(Y_test,predict))


print('Score',score)
print('Error',error)
print(list(zip(predict, Y_test))[:5])

Score 0.7528945521902535
Error 40705574.57565972
[(1501146941.65, 1489711738), (1486135724.55, 1481213177), (1505532971.55, 1502320366), (1488977574.65, 1503936106), (1329700428.3, 1294272000)]


In [568]:
dfx={}
dfx=pd.DataFrame(dfx)

#df['Fecha1']=X_test['Fecha1']
dfx['Y_test']=Y_test
dfx['Predict']=predict

#df['Fecha1']=pd.to_datetime(df['Fecha1'], unit='s')
dfx['Y_test']=pd.to_datetime(dfx['Y_test'], unit='s')
dfx['Predict']=pd.to_datetime(dfx['Predict'], unit='s')
dfx.head(10)

,Y_test,Predict
1011,2017-03-17 00:48:58,2017-07-27 09:15:41.650000095
447,2016-12-08 16:06:17,2017-02-03 15:28:44.549999952
908,2017-08-09 23:12:46,2017-09-16 03:36:11.549999952
809,2017-08-28 16:01:46,2017-03-08 12:52:54.650000095
1897,2011-01-06 00:00:00,2012-02-20 01:13:48.299999952
2734,2013-04-18 09:15:18,2014-06-30 17:22:17.500000000
3032,2015-02-05 19:30:58,2012-09-24 21:04:53.450000048
401,2018-03-13 00:02:35,2016-08-03 12:30:59.950000048
1743,2011-03-18 00:00:00,2012-08-02 13:36:42.099999905
1590,2015-08-20 17:55:32,2015-08-05 11:28:59.299999952
